In [1]:
# Imports required libraries
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import hvplot
from pathlib import Path

%matplotlib inline

In [2]:
load_dotenv()

True

In [3]:
# Locates and stores keys
alpaca_api = os.getenv("ALPACA_API_KEY")
alpaca_secret = os.getenv("ALPACA_SECRET_KEY")
type(alpaca_api)
type(alpaca_secret)

# Creates the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(
    alpaca_api,
    alpaca_secret,
    api_version="v2"
)

In [4]:
# Sets timeframe to 1Day
timeframe = "1Day"

# Formats current date as ISO format
# You can choose to set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
# Alternatively you could select any arbitrary start/end dates
start_date = pd.Timestamp("2022-05-12", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-05-12", tz="America/New_York").isoformat()

In [5]:
# Creates the list of tickers that the user will input, and allows for their input

tickers = []

tickers.append(input("Enter your first stock or bond ticker (as it appears on the exchange (example: BBBY)) "))
tickers.append(input("Enter your second ticker."))
tickers.append(input("Enter your third ticker."))
tickers.append(input("Enter your fourth ticker."))
tickers.append(input("Enter your fifth ticker."))
print(f"The tickers you have selected are: {tickers}")


Enter your first stock or bond ticker (as it appears on the exchange (example: BBBY))  GME
Enter your second ticker. AMC
Enter your third ticker. MSFT
Enter your fourth ticker. GOOG
Enter your fifth ticker. AMZN


The tickers you have selected are: ['GME', 'AMC', 'MSFT', 'GOOG', 'AMZN']


In [6]:
# Uses the Alpaca get_bars function to get current closing prices each ticker in
# the portfolio and creates a dataframe
portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start_date,
    end_date
).df
    
# Reorganizes the DataFrame
# Separates ticker data
ticker_1 = portfolio[portfolio['symbol']==portfolio.iloc[:,7]].drop('symbol', axis=1)
ticker_2 = portfolio[portfolio['symbol']==portfolio.iloc[:,7]].drop('symbol', axis=1)
ticker_3 = portfolio[portfolio['symbol']==portfolio.iloc[:,7]].drop('symbol', axis=1)
ticker_4 = portfolio[portfolio['symbol']==portfolio.iloc[:,7]].drop('symbol', axis=1)
ticker_5 = portfolio[portfolio['symbol']==portfolio.iloc[:,7]].drop('symbol', axis=1)

# Concatenates the ticker DataFrames
portfolio_df = pd.concat(
    [ticker_1, ticker_2, ticker_3, ticker_4, ticker_5],
    axis=1,
    keys=[tickers[0], tickers[1], tickers[2], tickers[3], tickers[4]]
)

# Reviews the Alpaca DataFrame
portfolio_df.head()

GME                                         \
                              open      high      low    close     volume   
timestamp                                                                   
2022-05-12 04:00:00+00:00    10.06    13.710     9.70    11.20  104735519   
2022-05-12 04:00:00+00:00  2055.00  2215.615  2048.11  2138.61    6574160   
2022-05-12 04:00:00+00:00    83.04   108.060    77.77    89.57   10023560   
2022-05-12 04:00:00+00:00  2240.00  2297.130  2202.27  2263.22    2055927   
2022-05-12 04:00:00+00:00   257.32   259.880   250.02   255.35   50675646   

                                                        AMC            \
                          trade_count         vwap     open      high   
timestamp                                                               
2022-05-12 04:00:00+00:00      508133    11.457367    10.06    13.710   
2022-05-12 04:00:00+00:00      358551  2129.391018  2055.00  2215.615   
2022-05-12 04:00:00+00:00      186854    91.085635    83.04   108.060   
2022-05-12 04:00:00+00:00      120731  2250.036001  2240.00  2297.130   
2022-05-12 04:00:00+00:00      709263   254.769493   257.32   259.880   

                                    ...       GOOG                           \
                               low  ...     volume trade_count         vwap   
timestamp                           ...                                       
2022-05-12 04:00:00+00:00     9.70  ...  104735519      508133    11.457367   
2022-05-12 04:00:00+00:00  2048.11  ...    6574160      358551  2129.391018   
2022-05-12 04:00:00+00:00    77.77  ...   10023560      186854    91.085635   
2022-05-12 04:00:00+00:00  2202.27  ...    2055927      120731  2250.036001   
2022-05-12 04:00:00+00:00   250.02  ...   50675646      709263   254.769493   

                              AMZN                                         \
                              open      high      low    close     volume   
timestamp                                                                   
2022-05-12 04:00:00+00:00    10.06    13.710     9.70    11.20  104735519   
2022-05-12 04:00:00+00:00  2055.00  2215.615  2048.11  2138.61    6574160   
2022-05-12 04:00:00+00:00    83.04   108.060    77.77    89.57   10023560   
2022-05-12 04:00:00+00:00  2240.00  2297.130  2202.27  2263.22    2055927   
2022-05-12 04:00:00+00:00   257.32   259.880   250.02   255.35   50675646   

                                                    
                          trade_count         vwap  
timestamp                                           
2022-05-12 04:00:00+00:00      508133    11.457367  
2022-05-12 04:00:00+00:00      358551  2129.391018  
2022-05-12 04:00:00+00:00      186854    91.085635  
2022-05-12 04:00:00+00:00      120731  2250.036001  
2022-05-12 04:00:00+00:00      709263   254.769493  

[5 rows x 35 columns]

In [7]:
# Allows the user to input the quantity of their stocks or bonds
ticker_1_qty = float(input(f"How many shares of {tickers[0]} do you own?"))
ticker_2_qty = float(input(f"How many shares of {tickers[1]} do you own?"))
ticker_3_qty = float(input(f"How many shares of {tickers[2]} do you own?"))
ticker_4_qty = float(input(f"How many shares of {tickers[3]} do you own?"))
ticker_5_qty = float(input(f"How many shares of {tickers[4]} do you own?"))


How many shares of GME do you own? 2
How many shares of AMC do you own? 3
How many shares of MSFT do you own? 4
How many shares of GOOG do you own? 5
How many shares of AMZN do you own? 6


In [8]:
# Fetches the closing prices of each ticker symbol in the portfolio
ticker_1_close = float(portfolio_df.iat[0,3])
ticker_2_close = float(portfolio_df.iat[1,3])
ticker_3_close = float(portfolio_df.iat[2,3])
ticker_4_close = float(portfolio_df.iat[3,3])
ticker_5_close = float(portfolio_df.iat[4,3])
                                    
print(f"The closing price of {tickers[0]} was {ticker_1_close}")
print(f"The closing price of {tickers[1]} was {ticker_2_close}")
print(f"The closing price of {tickers[2]} was {ticker_3_close}")
print(f"The closing price of {tickers[3]} was {ticker_4_close}")
print(f"The closing price of {tickers[4]} was {ticker_5_close}")

The closing price of GME was 11.2
The closing price of AMC was 2138.61
The closing price of MSFT was 89.57
The closing price of GOOG was 2263.22
The closing price of AMZN was 255.35


In [16]:
# Calculates the value of each ticker based on quantity owned
ticker_1_value = ticker_1_qty * ticker_1_close
ticker_2_value = ticker_2_qty * ticker_2_close
ticker_3_value = ticker_3_qty * ticker_3_close
ticker_4_value = ticker_4_qty * ticker_3_close
ticker_5_value = ticker_5_qty * ticker_3_close

print(ticker_1_value, ticker_2_value, ticker_3_value, ticker_4_value, ticker_5_value)

# Adds up value of entire portfolio
portfolio_value = (ticker_1_value + ticker_2_value + ticker_3_value + ticker_4_value + ticker_5_value)
print(f"Your portfolio value is ${portfolio_value}")

22.4 6415.83 358.28 447.84999999999997 537.42
Your portfolio value is $7781.78


In [10]:
# Reads the S&P 500 data into a dataframe
sp500 = pd.read_csv(
    Path("Resources/Download Data - INDEX_US_S&P US_SPX.csv"),
    index_col="Date",
    parse_dates=True,
    infer_datetime_format=True
)

In [11]:
# Displays the data
display(sp500.head())
display(sp500.tail())

,Open,High,Low,Close
Date,,,,
2022-05-10,"4,035.18","4,068.82","3,958.17","4,001.05"
2022-05-09,"4,081.27","4,081.27","3,975.48","3,991.24"
2022-05-06,"4,128.17","4,157.69","4,067.91","4,123.34"
2022-05-05,"4,270.43","4,270.43","4,106.01","4,146.87"
2022-05-04,"4,181.18","4,307.66","4,148.91","4,300.17"


,Open,High,Low,Close
Date,,,,
2021-05-14,"4,129.58","4,183.13","4,129.58","4,173.85"
2021-05-13,"4,074.99","4,131.58","4,074.99","4,112.50"
2021-05-12,"4,130.55","4,134.73","4,056.88","4,063.04"
2021-05-11,"4,150.34","4,162.04","4,111.53","4,152.10"
2021-05-10,"4,228.29","4,236.39","4,188.13","4,188.43"


In [12]:
# Checks for null values in the data
sp500.isnull().sum()

Open     0
High     0
Low      0
Close    0
dtype: int64

In [13]:
# Cleans and prepares the data for the dataframe
sp500.loc[:, 'Open'] = sp500.loc[:, 'Open'].str.replace(",", "")
sp500.loc[:, 'High'] = sp500.loc[:, 'High'].str.replace(",", "")
sp500.loc[:, 'Low'] = sp500.loc[:, 'Low'].str.replace(",", "")
sp500.loc[:, 'Close'] = sp500.loc[:, 'Close'].str.replace(",", "")

# Confirms cleaning of data
sp500.head()

,Open,High,Low,Close
Date,,,,
2022-05-10,4035.18,4068.82,3958.17,4001.05
2022-05-09,4081.27,4081.27,3975.48,3991.24
2022-05-06,4128.17,4157.69,4067.91,4123.34
2022-05-05,4270.43,4270.43,4106.01,4146.87
2022-05-04,4181.18,4307.66,4148.91,4300.17


In [14]:
# Changes all data types to float
sp500.loc[:, 'Open'] = sp500.loc[:, 'Open'].astype('float')
sp500.loc[:, 'High'] = sp500.loc[:, 'High'].astype('float')
sp500.loc[:, 'Low'] = sp500.loc[:, 'Low'].astype('float')
sp500.loc[:, 'Close'] = sp500.loc[:, 'Close'].astype('float')

# Confirms data type conversion
sp500.dtypes

Open     float64
High     float64
Low      float64
Close    float64
dtype: object